<a href="https://colab.research.google.com/github/NehaKumari500092077/Visual-Computing/blob/main/Assignment1_Task5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task 5: Bicubic Interpolation (6 Marks)
Upsample an image by a factor of 4 for width and 4 for height using bicubic interpolation. Implement the interpolation manually without using OpenCV's interpolation functions. Verify the result using OpenCV. Use a_1_task_5.png as input.
